# Working with Parallel Function Calls and Multiple Function Responses

Notebook is available at:
https://github.com/Bluedata-Consulting/GAAPB03-training

## Overview

Funciton Calling lets you create a description of a function in your code, then pass that description to a language model in a request. The response from the model includes the name of a function that matches the description and the arguments to call it with.

In this tutorial, you'll learn how to work with parallel function calling including:
    
- Handling parallel function calls for repeated functions
- Working with parallel function calls across multiple functions
- Extracting multiple function calls from a Gemini response
- Calling multiple functions and returning them to Gemini

### What is parallel function calling?

In previous versions of LLMs, it would return two or more chained function calls if the model determined that more than one function call was needed before returning a natural language summary. Here, a chained function call means that you get the first function call response, return the API data to Gemini, get a second function call response, return the API data to Gemini, and so on.

In recent versions of specific Language models (from May 2024 and on), most language models have the ability to return two or more function calls in parallel (i.e., two or more function call responses within the first function call response object). Parallel function calling allows you to fan out and parallelize your API calls or other actions that you perform in your application code, so you don't have to work through each function call response and return one-by-one!


<img src="https://storage.googleapis.com/github-repo/generative-ai/gemini/function-calling/parallel-function-calling-in-gemini.png">

In [1]:
!python -m pip install wikipedia openai --quiet

In [16]:
from openai import AzureOpenAI
client = AzureOpenAI()
model_name='gpt4o'

### Define helper function

In [17]:
def extract_tool_calls(response: dict):
    tool_calls = []
    tool_call = response.choices[0].message.tool_calls
    if tool_call:
        for tool in tool_call:
            function_name = tool.function.name
            function_args = eval(tool.function.arguments)  # Safely convert string to dictionary
            function_call_dict = {function_name: function_args}
            function_call_dict['id'] = tool.id
            tool_calls.append(function_call_dict)

    return tool_calls

## Example: Parallel function calls on the same function

A great use case for parallel function calling is when you have a function that only accepts one parameter per API call and you need to make repeated calls to that function.

With Parallel Function Calling, rather than having to send N number of API requests to LLM for N number function calls, instead you can send a single API request to LLM, receive N number of Function Call Responses within a single response, make N number of external API calls in your code, then return all of the API responses to LLM in bulk. And you can do all of this without any extra configuration in your function declarations, tools, or requests to LLM.

In this example, you'll do exactly that and use Parallel Function Calling in LLM to ask about multiple topics on [Wikipedia](https://www.wikipedia.org/). Let's get started!


In [18]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_wikipedia",
            "description": "Search for articles or any other relevant information on Wikipedia'",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Query to search for on Wikipedia"
                    }
                },
                "required": ["query"],
                "additionalProperties": False
            }
        }
    }
]

In [19]:
### Send prompt to OpenAI GPT-4o with function calling
prompt = "Search for articles related to solar panels, renewable energy, and battery storage and provide a summary of your findings"

In [20]:
response = client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    tools=tools,
    tool_choice="auto",
)


In [21]:
response.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_p8qeJ7i1dV4vDPNS8EXJtILP', function=Function(arguments='{"query": "solar panels"}', name='search_wikipedia'), type='function'),
 ChatCompletionMessageToolCall(id='call_rvqnhCLbI36er0VHTZS1nlIr', function=Function(arguments='{"query": "renewable energy"}', name='search_wikipedia'), type='function'),
 ChatCompletionMessageToolCall(id='call_ls6aWEexLmAc8cdTnSxCRb44', function=Function(arguments='{"query": "battery storage"}', name='search_wikipedia'), type='function')]

In [22]:
response2 = client.chat.completions.create(model=model_name,
                                           messages=[{"role":"user","content":"Define Artificial Intelligence"}],
                                           max_tokens=10,n=2)

print(response2.model_dump_json(indent=2))

{
  "id": "chatcmpl-CDhlQ1BmONotJDjImNhYs8av0PgSf",
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Artificial Intelligence (AI) refers to the simulation of",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null,
        "annotations": [],
        "refusal": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "protected_material_code": {
          "filtered": false,
          "detected": false
        },
        "protected_material_text": {
          "filtered": false,
          "detected": false
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
    

In [23]:
print(response.model_dump_json(indent=2))

{
  "id": "chatcmpl-CDhl9L7Im1R17wuXKizV88FdKbRwF",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "role": "assistant",
        "function_call": null,
        "tool_calls": [
          {
            "id": "call_p8qeJ7i1dV4vDPNS8EXJtILP",
            "function": {
              "arguments": "{\"query\": \"solar panels\"}",
              "name": "search_wikipedia"
            },
            "type": "function"
          },
          {
            "id": "call_rvqnhCLbI36er0VHTZS1nlIr",
            "function": {
              "arguments": "{\"query\": \"renewable energy\"}",
              "name": "search_wikipedia"
            },
            "type": "function"
          },
          {
            "id": "call_ls6aWEexLmAc8cdTnSxCRb44",
            "function": {
              "arguments": "{\"query\": \"battery storage\"}",
              "name": "search_wikipedia"
            },
          

In [24]:
# Extract function calls from the response
tool_calls = extract_tool_calls(response)
print(tool_calls)

[{'search_wikipedia': {'query': 'solar panels'}, 'id': 'call_p8qeJ7i1dV4vDPNS8EXJtILP'}, {'search_wikipedia': {'query': 'renewable energy'}, 'id': 'call_rvqnhCLbI36er0VHTZS1nlIr'}, {'search_wikipedia': {'query': 'battery storage'}, 'id': 'call_ls6aWEexLmAc8cdTnSxCRb44'}]


In [25]:
### Make external API calls
import wikipedia
api_response = []

# Loop over multiple function calls
for tool in tool_calls:
    print(tool)

    # Make external API call
    result = wikipedia.summary(tool["search_wikipedia"]["query"])

    # Collect all API responses
    api_response.append(result)

{'search_wikipedia': {'query': 'solar panels'}, 'id': 'call_p8qeJ7i1dV4vDPNS8EXJtILP'}
{'search_wikipedia': {'query': 'renewable energy'}, 'id': 'call_rvqnhCLbI36er0VHTZS1nlIr'}
{'search_wikipedia': {'query': 'battery storage'}, 'id': 'call_ls6aWEexLmAc8cdTnSxCRb44'}


In [26]:
for res in api_response:
    print(res)

A solar panel is a device that converts sunlight into electricity by using multiple solar modules that consist of photovoltaic (PV) cells. PV cells are made of materials that produce excited electrons when exposed to light. These electrons flow through a circuit and produce direct current (DC) electricity, which can be used to power various devices or be stored in batteries. Solar panels can be known as solar cell panels, or solar electric panels. Solar panels are usually arranged in groups called arrays or systems. A photovoltaic system consists of one or more solar panels, an inverter that converts DC electricity to alternating current (AC) electricity, and sometimes other components such as controllers, meters, and trackers. Most panels are in solar farms or rooftop solar panels which  supply the electricity grid.
Some advantages of solar panels are that they use a renewable and clean source of energy, reduce greenhouse gas emissions, and lower electricity bills. Some disadvantages 

In [27]:

### Generate a natural language summary
messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},]

messages.append(response.choices[0].message)
for tool,res in zip(tool_calls,api_response):
  messages.append({"tool_call_id": tool['id'],
                   "role": "tool",
                   "name": 'search_wikipedia',
                   "content": res,})


In [29]:
# Now we return all API responses back to GPT-4 to summarize

new_response = client.chat.completions.create(
    model=model_name,
    messages = messages)

In [30]:

# Display the final summary
print(new_response.choices[0].message.content)

Here are the summaries of the findings related to solar panels, renewable energy, and battery storage:

### Solar Panels
Solar panels consist of photovoltaic (PV) cells that convert sunlight into electricity. These panels form arrays or systems and are often paired with an inverter to convert the generated DC electricity to AC electricity. While solar panels can provide significant advantages such as being a renewable and clean energy source, reducing greenhouse gas emissions, and potentially lowering electricity bills, they do have drawbacks. These include their dependency on sunlight availability, the need for regular maintenance, and high initial costs. Solar panels are utilized in various applications, from residential rooftops to large solar farms, and they are often integrated with battery storage to enhance electricity reliability.

### Renewable Energy
Renewable energy is derived from naturally replenishing resources such as solar, wind, hydropower, bioenergy, and geothermal en

## Example: Parallel function calls across multiple functions

Another good fit for parallel function calling is when you have multiple, independent functions that you want to be able to call in parallel, which reduces the number of consecutive OpenAI API calls that you need to make and (ideally) reduces the overall response time to the end user who is waiting for a natural language response.

In this example, you'll use Parallel Function Calling in LLM to ask about multiple aspects of topics and articles on [Wikipedia](https://www.wikipedia.org/).


In [31]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_wikipedia",
            "description": "Search for articles or any other relevant information on Wikipedia",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Query to search for on Wikipedia"
                    }
                },
                "required": ["query"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "suggest_wikipedia",
            "description": "Get suggested titles from Wikipedia based on the given query term",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Query to search for suggested titles on Wikipedia"
                    }
                },
                "required": ["query"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "summarize_wikipedia",
            "description": "Retrieve brief summaries of Wikipedia articles related to a given topic",
            "parameters": {
                "type": "object",
                "properties": {
                    "topic": {
                        "type": "string",
                        "description": "Topic to search for article summaries on Wikipedia"
                    }
                },
                "required": ["topic"],
                "additionalProperties": False
            }
        }
    }
]


In [32]:
prompt = "Show the search results, variations, and article summaries about Wikipedia articles related to the solar system"

In [34]:
response = client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    tools=tools,
    tool_choice='auto'
)


In [35]:
response.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_1tnFUFVQpjoxEJSe5buAybca', function=Function(arguments='{"query": "solar system"}', name='search_wikipedia'), type='function'),
 ChatCompletionMessageToolCall(id='call_v18lXNqcMTLqVgIO4Y7gTkjW', function=Function(arguments='{"query": "solar system"}', name='suggest_wikipedia'), type='function'),
 ChatCompletionMessageToolCall(id='call_mhC0fdG6Lk9cZLCLKqeU1xQL', function=Function(arguments='{"topic": "solar system"}', name='summarize_wikipedia'), type='function')]

### Extract function names and parameters

Use the helper function that we created earlier to extract the function names and function parameters for each Function Call that LLM responded with:

In [36]:
# Extract function calls from the response
tool_calls = extract_tool_calls(response)
print(tool_calls)

[{'search_wikipedia': {'query': 'solar system'}, 'id': 'call_1tnFUFVQpjoxEJSe5buAybca'}, {'suggest_wikipedia': {'query': 'solar system'}, 'id': 'call_v18lXNqcMTLqVgIO4Y7gTkjW'}, {'summarize_wikipedia': {'topic': 'solar system'}, 'id': 'call_mhC0fdG6Lk9cZLCLKqeU1xQL'}]


### Make external API calls

Next, you'll loop through the Function Calls and use the `wikipedia` Python package to make APIs calls and gather information from Wikipedia:

In [37]:
api_response = {}

# Loop over multiple function calls
for tool in tool_calls:
    # Extract the function name
    print(tool)
    function_name = list(tool.keys())[0]

    # Determine which external API call to make
    if function_name == "search_wikipedia":
        result = wikipedia.search(tool["search_wikipedia"]["query"])
    if function_name == "suggest_wikipedia":
        result = wikipedia.suggest(tool["suggest_wikipedia"]["query"])
    if function_name == "summarize_wikipedia":
        result = wikipedia.summary(
            tool["summarize_wikipedia"]["topic"], auto_suggest=False
        )

    # Collect all API responses
    api_response[function_name] = result

{'search_wikipedia': {'query': 'solar system'}, 'id': 'call_1tnFUFVQpjoxEJSe5buAybca'}
{'suggest_wikipedia': {'query': 'solar system'}, 'id': 'call_v18lXNqcMTLqVgIO4Y7gTkjW'}
{'summarize_wikipedia': {'topic': 'solar system'}, 'id': 'call_mhC0fdG6Lk9cZLCLKqeU1xQL'}


In [38]:
api_response

{'search_wikipedia': ['Solar System',
  'Formation and evolution of the Solar System',
  'List of Solar System objects by size',
  'List of Solar System objects',
  'Photovoltaic system',
  'Exoplanet',
  'Solar System model',
  'Solar System (disambiguation)',
  'Passive solar building design',
  'Solar water heating'],
 'suggest_wikipedia': 'soler system',
 'summarize_wikipedia': "The Solar System consists of the Sun and the objects that orbit it. The name comes from Sōl, the Latin name for the Sun. It formed about 4.6 billion years ago when a dense region of a molecular cloud collapsed, creating the Sun and a protoplanetary disc from which the orbiting bodies assembled. The fusion of hydrogen into helium inside the Sun's core releases energy, which is primarily emitted through its outer photosphere. This creates a decreasing temperature gradient across the system. Over 99.86% of the Solar System's mass is located within the Sun.\nThe most massive objects that orbit the Sun are the e

### Get a natural language summary

Now you can return all of the API responses to LLM so that it can generate a natural language summary:

In [39]:

### Generate a natural language summary
messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},]

messages.append(response.choices[0].message)
for tool,res in zip(tool_calls,api_response):
  function_name = list(tool.keys())[0]
  messages.append({"tool_call_id": tool['id'],
                   "role": "tool",
                   "name": function_name,
                   "content": res,})


In [41]:
# Now we return all API responses back to GPT-4 to summarize

new_response = client.chat.completions.create(
    model=model_name,
    messages = messages)

In [42]:
# Display the final summary
print(new_response.choices[0].message.content)

Here are the search results, variations, and a summary related to Wikipedia articles on the solar system:

### Search Results
1. **Solar System** - A gravitationally bound system comprising the Sun and the objects that orbit it, including the eight planets, their moons, dwarf planets such as Pluto, and other small solar system bodies.
2. **Planet (A body of significant mass)** - Defines what a planet is, including classifications and examples from the solar system.
3. **Earth** - Our home planet, the third from the Sun, known for its biodiversity and liquid water.
4. **Mars** - The fourth planet from the Sun, known as the Red Planet due to iron oxide on its surface.
5. **Jupiter** - The largest planet in the solar system, a gas giant with a prominent set of rings and many moons.
6. **Saturn** - Known for its stunning ring system, it is the second-largest planet in the solar system and a gas giant.
7. **Dwarf planet** - Discusses celestial objects like Pluto that don't meet all criteria

# Sequential Multi function call

In [43]:
import requests
import json

In [44]:
# Define the OpenWeatherMap API key
OWM_API_KEY = "29af1cea50a401d8e624eea4660b3f59"

def get_current_weather(location, unit="kelvin"):
    """
    Fetches the current weather information for a given location.

    Parameters:
    - location: str, the name of the location (e.g., "Paris").
    - unit: str, the unit of temperature (default is "kelvin").

    Returns:
    - str: JSON formatted string containing weather information.
    """
    # Construct the API request URL
    url = f"https://api.openweathermap.org/data/2.5/weather?q={location}&appid={OWM_API_KEY}"

    try:
      # Send the API request
      response = requests.get(url)
    except:
      return "Error occurred because location does not exist"

    # Parse the temperature and weather forecast from the response
    temp = response.json()['main']['temp']
    forecast = [response.json()['weather'][0]['main'], response.json()['weather'][0]['description']]

    # Create a dictionary with the weather information
    weather_info = {
        "location": location,
        "temperature": temp,
        "unit": 'Kelvin',
        "forecast": forecast
    }

    # Return the weather information as a JSON string
    return json.dumps(weather_info)

In [45]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_wikipedia",
            "description": "Search for articles or any other relevant information on Wikipedia'",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Query to search for on Wikipedia"
                    }
                },
                "required": ["query"],
                "additionalProperties": False
            }
        }
    },
    {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit","kelvin"]},
                    },
                    "required": ["location"],
                },
            },
        },
]

In [46]:
prompt = "Get the name of capital city of Tripura using wikipedia and then tell me the weather information for the same using wikipedia results. "

In [48]:
response = client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    tools=tools,
    tool_choice='auto'
)


In [49]:
response.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_mEgbQ5VQnbZcBz7GMSsQxFh6', function=Function(arguments='{"query":"Tripura"}', name='search_wikipedia'), type='function')]

### Extract function names and parameters

Use the helper function that we created earlier to extract the function names and function parameters for each Function Call that LLM responded with:

In [50]:
# Extract function calls from the response
tool_calls = extract_tool_calls(response)
print(tool_calls)

[{'search_wikipedia': {'query': 'Tripura'}, 'id': 'call_mEgbQ5VQnbZcBz7GMSsQxFh6'}]


In [51]:
### Make external API calls
import wikipedia
api_response = []

# Loop over multiple function calls
for tool in tool_calls:
    print(tool)

    # Make external API call
    result = wikipedia.summary(tool["search_wikipedia"]["query"])

    # Collect all API responses
    api_response.append(result)

{'search_wikipedia': {'query': 'Tripura'}, 'id': 'call_mEgbQ5VQnbZcBz7GMSsQxFh6'}


In [52]:
for res in api_response:
    print(res)

Tripura () is a state in northeastern India. The third-smallest state in the country, it covers 10,491 km2 (4,051 sq mi); and the seventh-least populous state with a population of 3.67 million. It is bordered by Assam and Mizoram to the east and by Bangladesh to the north, south and west. Tripura is divided into 8 districts and 23 sub-divisions, where Agartala is the capital and the largest city in the state. Tripura has 19 different tribal communities with a majority Bengali population. Kokborok, Bengali, and English are the state's official languages.
The area of modern Tripura — ruled for several centuries by the Manikya Dynasty — was part of the Tripuri Kingdom (also known as Hill Tippera). It became a princely state under the British Raj during its tenure, and acceded to independent India in 1947. It merged with India in 1949 and was designated as a 'Part C State' (union territory). It became a full-fledged state of India in 1972.
Tripura lies in a geographically isolated location

In [53]:
### Generate a natural language summary
messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},]

messages.append(response.choices[0].message)
for tool,res in zip(tool_calls,api_response):
  messages.append({"tool_call_id": tool['id'],
                   "role": "tool",
                   "name": 'search_wikipedia',
                   "content": res,})


In [54]:
# Now we return all API responses back to GPT-4 to summarize

new_response = client.chat.completions.create(
    model=model_name,
    messages = messages,
    tools=tools,
    tool_choice='auto')

In [55]:
# Display the final summary
print(new_response.model_dump_json(indent=2))

{
  "id": "chatcmpl-CDhnpSa4oXl46i6zcNFzOZAsEWnz9",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "role": "assistant",
        "function_call": null,
        "tool_calls": [
          {
            "id": "call_jfajvSJn7MMD4J5fjN88M0Km",
            "function": {
              "arguments": "{\"location\":\"Agartala, Tripura\",\"unit\":\"celsius\"}",
              "name": "get_current_weather"
            },
            "type": "function"
          }
        ],
        "annotations": [],
        "refusal": null
      },
      "content_filter_results": {}
    }
  ],
  "created": 1757382677,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_efad92c60b",
  "usage": {
    "completion_tokens": 26,
    "prompt_tokens": 696,
    "total_tokens": 722,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0